### MANDATORY ASSIGNMENT 2

In [1]:
from sklearn import datasets

In [2]:
import numpy as np

In [3]:
iris = datasets.load_iris()

In [4]:
X = iris.data
Y = iris.target

#### Task 1) data exploration

In [5]:
len(X)

150

In [6]:
print(X.shape, Y.shape)

(150, 4) (150,)


In [7]:
print(np.min(X), np.max(X))
print(np.min(Y), np.max(Y))

0.1 7.9
0 2


##### 2) Encoding and pre-processing

In [93]:
#at first we normalize the data from 0 to pi, and then implement angle encoding
from sklearn.preprocessing import MinMaxScaler
from qiskit import QuantumCircuit, transpile, assemble
from qiskit_aer import Aer, AerSimulator
from qiskit.visualization import plot_histogram
from qiskit.circuit import Parameter
from qiskit_algorithms.optimizers import SPSA
import random

In [9]:
scaler = MinMaxScaler(feature_range=(0, np.pi))
X = scaler.fit_transform(X)

In [10]:
def angle_encoding(qc, sample):
    for qubit in range(len(qc.qubits)):
        qc.rx(sample[qubit], qubit)

    

##### 3) choosing Loss function

In [20]:
from sklearn.metrics import root_mean_squared_error


##### 4) splitting data

In [12]:
from sklearn.model_selection import train_test_split

In [13]:
X_train, X_temp, y_train, y_temp = train_test_split(X, Y, test_size=0.3, random_state=42) # 70% training 
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42) # 15% validation, 15% testing

In [14]:
num_qubits = 4
num_layers = 3

In [15]:
int('1111', 2)

15

In [16]:
theta = '\u03B8'

def real_amplitudes(data_point,parameters, layers = num_layers):
    qc = QuantumCircuit(num_qubits)
    angle_encoding(qc, data_point)

    param_index = 0

    for layer in range(layers):
        for qubit in range(len(qc.qubits)):
            qc.ry(parameters[param_index], qubit)
            param_index += 1
        qc.barrier()
        
        for qubit in range(len(qc.qubits)-1):
            qc.cx(qubit, qubit+1)
        qc.barrier()

    return qc
    

In [17]:
def data_decoding(output):
    return int(output, 2) % 3 

In [127]:
def objective_function(updated_params):
    loss = 0
    transpilation_cache = {}
    backend = AerSimulator(method = 'statevector')

    for x, y in zip(X_train, y_train):
        qc = real_amplitudes(x, updated_params, layers=num_layers)
        qc.measure_all()
        shots = 100

        circuit_key = tuple(x)  # Convert input to immutable tuple for dict key
        if circuit_key not in transpilation_cache:
            transpiled_qc = transpile(qc, backend)  # Done only for new circuits
            transpilation_cache[circuit_key] = transpiled_qc

        tqc = transpilation_cache[circuit_key]
        
        job = backend.run(tqc, shots=shots)
        result = job.result()
        counts = result.get_counts(qc)

        count_classes = {0: 0, 1: 0, 2: 0}
        for output, count in counts.items():
            class_num = data_decoding(output)
            count_classes[class_num] += count

        expectation_value = 0    
        for class_num, count in count_classes.items():
            probability = count / shots
            expectation_value += class_num * probability
        

        loss += np.sqrt((y - expectation_value) ** 2)
        
     
    loss = loss / len(X_train)

    print(f"Parameters: {updated_params} loss: {loss}")
    return loss
    

In [128]:
y_train[0:10]

array([1, 2, 2, 1, 2, 1, 2, 1, 0, 2])

In [ ]:
initial_parameters = np.random.rand(num_layers * num_qubits)

# Gradient Descent optimizer
optimizer = SPSA(maxiter=50)

# Optimize the parameters
optimized = optimizer.minimize(fun=objective_function, x0=initial_parameters)

print("Optimized Parameters:", optimized.x)
print("Minimum Cost:", optimized.fun)


Parameters: [ 0.40965767  0.12046764  1.14698224  1.02737926  0.620943    0.61949672
  0.42512417  0.50856737 -0.11277331  0.91535068  0.47820765  0.44227581] loss: 0.7626666666666666
Parameters: [0.00965767 0.52046764 0.74698224 0.62737926 1.020943   1.01949672
 0.02512417 0.10856737 0.28722669 0.51535068 0.07820765 0.84227581] loss: 0.6789523809523808
Parameters: [ 0.40965767  0.52046764  1.14698224  0.62737926  0.620943    0.61949672
  0.42512417  0.50856737 -0.11277331  0.51535068  0.07820765  0.44227581] loss: 0.6482857142857141
Parameters: [0.00965767 0.12046764 0.74698224 1.02737926 1.020943   1.01949672
 0.02512417 0.10856737 0.28722669 0.91535068 0.47820765 0.84227581] loss: 0.7856190476190477
Parameters: [ 0.40965767  0.52046764  0.74698224  0.62737926  0.620943    1.01949672
  0.02512417  0.10856737 -0.11277331  0.51535068  0.47820765  0.44227581] loss: 0.6786666666666669
Parameters: [0.00965767 0.12046764 1.14698224 1.02737926 1.020943   0.61949672
 0.42512417 0.50856737 0.

In [115]:
def predict(data_point, optimized_params):
    qc = real_amplitudes(data_point, optimized_params)
    qc.measure_all()

    backend = AerSimulator(method = 'statevector')
    tqc = transpile(qc, backend)
    shots = 100
    job = backend.run(tqc, shots=shots)
    result = job.result()
    counts = result.get_counts(qc)

    count_classes = {0: 0, 1: 0, 2: 0}
        
    # Decode each measurement outcome and aggregate counts for each class
    for output, count in counts.items():
        class_num = data_decoding(output)
        count_classes[class_num] += count
    
    # Calculate probabilities for each class
    probabilities = {class_num: count / shots for class_num, count in count_classes.items()}
    
    # Determine the predicted class by choosing the class with the highest probability
    predicted_class = max(probabilities, key=probabilities.get)
    
    return predicted_class

In [116]:
def predict_dataset(X, optimized_params):
    return [predict(data_point, optimized_params) for data_point in X]

In [117]:
predictions = predict_dataset(X_test, optimized.x)
predictions

[0, 2, 2, 0, 2, 2, 2, 0, 2, 2, 0, 1, 2, 2, 0, 0, 2, 2, 0, 2, 0, 0, 2]

In [118]:
from sklearn.metrics import accuracy_score

In [119]:
score = accuracy_score(y_test, predictions)
score

0.5217391304347826

In [120]:
y_test

array([0, 2, 2, 0, 2, 1, 1, 0, 1, 1, 1, 2, 1, 2, 1, 0, 1, 1, 1, 2, 0, 0,
       2])